In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os 

## 1.Get all links for zimuku items

In [2]:
## send the request
url = "http://www.zimuku.net/search?q=&t=onlyst"
result = requests.get(url)
print(result.status_code)

200


In [3]:
## get page number
c = result.content
soup = BeautifulSoup(c,"lxml")
pages =  soup.find('div',{'class':'pagination r clearfix'}).find('div')
page_num = pages.findAll('a')[-1].getText()
page_num = int(page_num)
print(page_num)


12993


In [4]:
## get download link from title link 
def get_down_link(url):
    #url = "http://www.zimuku.net/shooter/57265.html"
    try:
        result = requests.get(url)
        if result.status_code != 200:
            return None
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        down_link =  soup.find('li',{'class':'li dlsub'}).find('a').get('href')
        return down_link
    except:
        return None

In [5]:
## download 
def download_zip(url,folder):
    try:
        result = requests.get(url)
        fname = result.headers['Content-Disposition'].split('"')[1]
        res= result.content
        with open( folder + fname, "wb" ) as f :
                f.write( res )
        return True
    except:
        return False

### set up some global variables

In [6]:
## create doc_list to hold all links 
doc_list = {'title':[],'link':[],'status':[]}
page_num = 1000                                ## process 1, 1 to 1000 pages
folder = 'data/process1/data_download/'

## loop through each page
for p in range(1,page_num+1):
    print(p-1, 'success')
    try:
        page_url = url + '&p=' + str(p)
        result = requests.get(page_url)
        #print(result.status_code)
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        docs = soup.find('div',{'class':'box clearfix'}).findAll('div',{'class':'persub clearfix'})

        for doc in docs:

            ## get title 
            link_tag = doc.find('h1',{'class':'title'}).find('a')
            title = link_tag.getText().strip(' \t\n\r@').replace('\n','').replace('\r',' ')
            if title == '_':
                continue
            elif "/" in title:
                 title = title.split("/")[0]

            link='http://www.zimuku.net'+link_tag.get('href')

            ## get download link 
            down_link = get_down_link(link)

            doc_list['title'].append(title)
            doc_list['link'].append(down_link)
        
            ## download file 
            try:
                response = download_zip(down_link,folder)
                doc_list['status'].append(response)
            except:
                doc_list['status'].append(False)
    except:
        continue

print(len(doc_list['link']))

0 success
1 success
2 success
3 success
4 success
5 success
6 success
7 success
8 success
9 success
10 success
11 success
12 success
13 success
14 success
15 success
16 success
17 success
18 success
19 success
20 success
21 success
22 success
23 success
24 success
25 success
26 success
27 success
28 success
29 success
30 success
31 success
32 success
33 success
34 success
35 success
36 success
37 success
38 success
39 success
40 success
41 success
42 success
43 success
44 success
45 success
46 success
47 success
48 success
49 success
50 success
51 success
52 success
53 success
54 success
55 success
56 success
57 success
58 success
59 success
60 success
61 success
62 success
63 success
64 success
65 success
66 success
67 success
68 success
69 success
70 success
71 success
72 success
73 success
74 success
75 success
76 success
77 success
78 success
79 success
80 success
81 success
82 success
83 success
84 success
85 success
86 success
87 success
88 success
89 success
90 success
91 succes

In [7]:
## export links file 
df = pd.DataFrame(doc_list)
df.to_csv('zimuku_links_v1.csv')
df.ix[0:5]

ValueError: arrays must all be same length

## Now start downloading